In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

from sktime.regression.kernel_based import RocketRegressor
import pickle

from apply_rocket import apply_rocket
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

In [3]:
with open('../datasets/training_dictionary.pkl', 'rb') as f:
    train_dict = pickle.load(f)

with open('../datasets/validation_dictionary.pkl', 'rb') as f:
    val_dict = pickle.load(f)

In [ ]:
i_position = 'FWD'
y_pred, y_val = apply_rocket(train_dict, val_dict, position=i_position, rocket_model='rocket')

mae = mean_absolute_error(y_val, y_pred)
rMAE = root_mean_squared_error(y_val, y_pred)
print(f'RMSE {i_position}: {rMAE}')
print(f'MAE {i_position}: {mae}')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Generating kernels with lengths: [ 7  9 11]
RMSE FWD: 33.93719973956141
MSE FWD: 29.584283959902134


In [ ]:
i_position = 'GK'
y_pred, y_val = apply_rocket(train_dict, val_dict, position=i_position, rocket_model='rocket')

mae = mean_absolute_error(y_val, y_pred)
rmse = root_mean_squared_error(y_val, y_pred)
print(f'RMSE {i_position}: {rmse}')
print(f'MAE {i_position}: {mae}')

Generating kernels with lengths: [ 7  9 11]
RMSE GK: 39.76132919563875
MSE GK: 35.175467379520825


In [ ]:
i_position = 'MID'
y_pred, y_val = apply_rocket(train_dict, val_dict, position=i_position, rocket_model='rocket')

mae = mean_absolute_error(y_val, y_pred)
rmse = root_mean_squared_error(y_val, y_pred)
print(f'RMSE {i_position}: {rmse}')
print(f'MAE {i_position}: {mae}')

Generating kernels with lengths: [ 7  9 11]


In [ ]:
i_position = 'DEF'
y_pred, y_val = apply_rocket(train_dict, val_dict, position=i_position, rocket_model='rocket')

mae = mean_absolute_error(y_val, y_pred)
rmse = root_mean_squared_error(y_val, y_pred)
print(f'RMSE {i_position}: {rmse}')
print(f'MAE {i_position}: {mae}')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Generating kernels with lengths: [ 7  9 11]
RMSE DEF: 40.639959605935594
MSE DEF: 38.855616865053236
